In [5]:
# Useful additional packages 
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from math import pi


from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import BasicAer, execute
from qiskit.quantum_info import Pauli, state_fidelity, basis_state, process_fidelity 

from qiskit.circuit import Parameter


from qiskit import *
IBMQ.load_accounts(hub=None)
from qiskit.tools.monitor import job_monitor, backend_monitor, backend_overview
from qiskit.providers.ibmq import least_busy

shots = 8192 
max_credits = 3 
#job_exp = execute(qc, backend=backend, shots=shots, max_credits=max_credits)
from qiskit.compiler import transpile

/Users/alicepagano/anaconda3/envs/Qiskitenv/lib/python3.6/site-packages/qiskit/providers/ibmq/ibmqprovider.py:272: UserWarning: Credentials are already in use.
  warnings.warn('Credentials are already in use.')


In [2]:
#Run in real quantum computer
#Misura di tutti i qubit
n = 5
q = QuantumRegister(n, 'q')
c = ClassicalRegister(n, 'c') #n

circ = QuantumCircuit(q,c)
phi = Parameter('phi')

circ.h(q[0])

for i in range(n-1):
    circ.cx(i, i+1)
    
circ.barrier()
circ.rz(phi, range(n))
circ.barrier()

for i in reversed(range(n-1)):
    circ.cx(i, i+1)

circ.h(q[0])

circ.barrier()

for i in range(n):
    circ.measure(q[i], c[i]) 

#circ.draw(output='mpl')

print('Phi range')
phi_range = [np.pi*j/(n+1) for j in range(2*n+1+1)]
print(phi_range)
circuits = [circ.bind_parameters({phi: phi_val}) for phi_val in phi_range] 
#print(circuits[3].parameters)
#circuits[11].draw(output='mpl')


job = execute(circ,
            backend=IBMQ.get_backend('ibmq_16_melbourne'),
            parameter_binds=[{phi: phi_val}
              for phi_val in phi_range],shots=shots)

job_monitor(job)

phi_range2 = phi_range
counts2 =[job.result().get_counts(i) for i in range(len(job.result().results))] 

print('\n')
print('Counts')
print(counts2)
print('\n')
print('Counts of 00000')
print(list(map(lambda c: 1/shots*c.get('00000', 0), counts2)))
job_id = job.job_id()
print('JOB ID: {}'.format(job_id))

Phi range
[0.0, 0.5235987755982988, 1.0471975511965976, 1.5707963267948966, 2.0943951023931953, 2.6179938779914944, 3.141592653589793, 3.665191429188092, 4.1887902047863905, 4.71238898038469, 5.235987755982989, 5.759586531581287]
Job Status: job has successfully run


Counts
[{'10011': 142, '00001': 380, '11111': 36, '01110': 69, '10001': 351, '01101': 102, '10000': 1070, '00110': 137, '10010': 193, '10111': 75, '10110': 115, '10101': 182, '01011': 57, '01001': 168, '11100': 280, '11011': 66, '00101': 193, '11010': 94, '00000': 1096, '01100': 269, '11001': 165, '11000': 486, '00100': 622, '00111': 74, '00010': 231, '10100': 594, '01111': 37, '00011': 171, '11110': 62, '01010': 101, '01000': 476, '11101': 98}, {'10011': 168, '00001': 390, '11111': 51, '01110': 46, '10001': 367, '01101': 188, '10000': 1085, '00110': 108, '10010': 212, '10111': 87, '10110': 111, '10101': 346, '01011': 60, '01001': 199, '11100': 171, '11011': 76, '00101': 353, '11010': 77, '00000': 1226, '01100': 154, '110

In [3]:
#Run in real quantum computer - Transpile
#Misura di tutti i qubit
n = 5
q = QuantumRegister(n, 'q')
c = ClassicalRegister(n, 'c') #n

circ = QuantumCircuit(q,c)
phi = Parameter('phi')

circ.h(q[0])

for i in range(n-1):
    circ.cx(i, i+1)
    
circ.barrier()
circ.rz(phi, range(n))
circ.barrier()

for i in reversed(range(n-1)):
    circ.cx(i, i+1)

circ.h(q[0])

circ.barrier()

for i in range(n):
    circ.measure(q[i], c[i]) 

#circ.draw(output='mpl')

print('Phi range')
phi_range = [np.pi*j/(n+1) for j in range(2*n+1+1)]
print(phi_range)
circuits = [circ.bind_parameters({phi: phi_val}) for phi_val in phi_range] 
#print(circuits[3].parameters)
#circuits[11].draw(output='mpl')

backend = IBMQ.get_backend('ibmq_16_melbourne')
trans_ghz = transpile(circ, backend)

job = execute(circ,
            backend,
            parameter_binds=[{phi: phi_val}
              for phi_val in phi_range],shots=shots)

job_monitor(job)

phi_range2 = phi_range
counts2 =[job.result().get_counts(i) for i in range(len(job.result().results))] 

print('\n')
print('Counts')
print(counts2)
print('\n')
print('Counts of 00000')
print(list(map(lambda c: 1/shots*c.get('00000', 0), counts2)))

job_id = job.job_id()
print('JOB ID: {}'.format(job_id))

#retrieved_job = backend.retrieve_job(job_id)
#retrieved_job.result().get_counts(qc)

Phi range
[0.0, 0.5235987755982988, 1.0471975511965976, 1.5707963267948966, 2.0943951023931953, 2.6179938779914944, 3.141592653589793, 3.665191429188092, 4.1887902047863905, 4.71238898038469, 5.235987755982989, 5.759586531581287]
Job Status: job has successfully run


Counts
[{'10011': 137, '00001': 417, '11111': 37, '01110': 50, '10001': 392, '01101': 115, '10000': 1110, '00110': 106, '10010': 188, '10111': 62, '10110': 107, '10101': 211, '01011': 72, '01001': 221, '11100': 270, '11011': 72, '00101': 193, '11010': 98, '00000': 1073, '01100': 211, '11001': 222, '11000': 550, '00100': 549, '00111': 67, '00010': 204, '10100': 573, '01111': 45, '00011': 143, '11110': 55, '01010': 91, '01000': 430, '11101': 121}, {'10011': 153, '00001': 471, '11111': 36, '01110': 48, '10001': 442, '01101': 184, '10000': 1081, '00110': 87, '10010': 180, '10111': 77, '10110': 102, '10101': 344, '01011': 68, '01001': 205, '11100': 180, '11011': 73, '00101': 361, '11010': 76, '00000': 1185, '01100': 159, '1100

In [9]:
#Run in real quantum computer - Transpile user
#Misura di tutti i qubit
n = 5
q = QuantumRegister(n, 'q')
c = ClassicalRegister(n, 'c') #n

circ = QuantumCircuit(q,c)
phi = Parameter('phi')

circ.h(q[0])

for i in range(n-1):
    circ.cx(i, i+1)
    
circ.barrier()
circ.rz(phi, range(n))
circ.barrier()

for i in reversed(range(n-1)):
    circ.cx(i, i+1)

circ.h(q[0])

circ.barrier()

for i in range(n):
    circ.measure(q[i], c[i]) 

#circ.draw(output='mpl')

print('Phi range')
phi_range = [np.pi*j/(n+1) for j in range(2*n+1+1)]
print(phi_range)
circuits = [circ.bind_parameters({phi: phi_val}) for phi_val in phi_range] 
#print(circuits[3].parameters)
#circuits[11].draw(output='mpl')

layout = {q[0]: 13, q[1]: 12, q[2]: 2, q[3]:3, q[4]:11}

backend = IBMQ.get_backend('ibmq_16_melbourne')
trans_ghz_mapped = transpile(circ, backend, initial_layout=layout)

job = execute(circ,
            backend,
            parameter_binds=[{phi: phi_val}
              for phi_val in phi_range],shots=shots)

job_monitor(job)

phi_range2 = phi_range
counts2 =[job.result().get_counts(i) for i in range(len(job.result().results))] 

print('\n')
print('Counts')
print(counts2)
print('\n')
print('Counts of 00000')
print(list(map(lambda c: 1/shots*c.get('00000', 0), counts2)))

job_id = job.job_id()
print('JOB ID: {}'.format(job_id))

#retrieved_job = backend.retrieve_job(job_id)
#retrieved_job.result().get_counts(qc)

Phi range
[0.0, 0.5235987755982988, 1.0471975511965976, 1.5707963267948966, 2.0943951023931953, 2.6179938779914944, 3.141592653589793, 3.665191429188092, 4.1887902047863905, 4.71238898038469, 5.235987755982989, 5.759586531581287]
Job Status: job has successfully run


Counts
[{'10011': 220, '00001': 1137, '11111': 29, '01110': 27, '10001': 1119, '01101': 70, '10000': 1028, '00110': 52, '10010': 217, '10111': 62, '10110': 69, '10101': 119, '01011': 83, '01001': 380, '11100': 91, '11011': 84, '00101': 112, '11010': 89, '00000': 1051, '01100': 72, '11001': 355, '11000': 375, '00100': 131, '00111': 49, '00010': 244, '10100': 140, '01111': 20, '00011': 210, '11110': 25, '01010': 75, '01000': 374, '11101': 83}, {'10011': 173, '00001': 917, '11111': 25, '01110': 33, '10001': 891, '01101': 55, '10000': 1329, '00110': 49, '10010': 221, '10111': 43, '10110': 52, '10101': 106, '01011': 75, '01001': 321, '11100': 92, '11011': 83, '00101': 109, '11010': 82, '00000': 1302, '01100': 93, '11001': 295,

In [8]:
n = 5
q = QuantumRegister(n, 'q')
c = ClassicalRegister(n, 'c') #n

circ = QuantumCircuit(q,c)
phi = Parameter('phi')

circ.h(q[0])

for i in range(n-1):
    circ.cx(i, i+1)
    
circ.barrier()
circ.rz(phi, range(n))
circ.barrier()

for i in reversed(range(n-1)):
    circ.cx(i, i+1)

circ.h(q[0])

circ.barrier()

for i in range(n):
    circ.measure(q[i], c[i]) 

#circ.draw(output='mpl')

print('Phi range')
phi_range = [np.pi*j/(n+1) for j in range(2*n+1+1)]
print(phi_range)
circuits = [circ.bind_parameters({phi: phi_val}) for phi_val in phi_range] 
#print(circuits[3].parameters)
#circuits[11].draw(output='mpl')

layout = {q[0]: 13, q[1]: 12, q[2]: 2, q[3]:3, q[4]:11}

backend = IBMQ.get_backend('ibmq_16_melbourne')
trans_ghz_mapped = transpile(circ, backend, initial_layout=layout)

trans_ghz = transpile(circ, backend)
print(circ.depth())
print(trans_ghz.depth())
print(trans_ghz_mapped.depth())

Phi range
[0.0, 0.5235987755982988, 1.0471975511965976, 1.5707963267948966, 2.0943951023931953, 2.6179938779914944, 3.141592653589793, 3.665191429188092, 4.1887902047863905, 4.71238898038469, 5.235987755982989, 5.759586531581287]
12
18
16


In [6]:
#Run in real quantum computer - Transpile user CIRCUITO MODIFICATOOOOO
#Misura di tutti i qubit
n = 5
q = QuantumRegister(n, 'q')
c = ClassicalRegister(n, 'c') #n

circ = QuantumCircuit(q,c)
phi = Parameter('phi')

#1 circuito modificato
circ.h(q[1])
circ.cx(q[1], q[0])
circ.cx(q[1], q[2])
circ.cx(q[2], q[3])
circ.cx(q[3], q[4])  
circ.barrier()
circ.rz(phi, range(n))
circ.barrier()
circ.cx(q[3], q[4])
circ.cx(q[2], q[3])
circ.cx(q[1], q[2])
circ.cx(q[1], q[0])
circ.h(q[1])

#2 circuito modificato
#circ.h(q[0])
#circ.cx(q[2], q[3])
#circ.cx(q[3], q[4])
#circ.cx(q[2], q[1])
#circ.cx(q[1], q[0])
    
#circ.barrier()
#circ.rz(phi, range(n))
#circ.barrier()

#circ.cx(q[1], q[0])
#circ.cx(q[2], q[1])
#circ.cx(q[3], q[4])
#circ.cx(q[2], q[3])
#circ.h(q[0])

circ.barrier()

for i in range(n):
    circ.measure(q[i], c[i]) 

#circ.draw(output='mpl')

print('Phi range')
phi_range = [np.pi*j/(n+1) for j in range(2*n+1+1)]
print(phi_range)
circuits = [circ.bind_parameters({phi: phi_val}) for phi_val in phi_range] 
#print(circuits[3].parameters)
#circuits[11].draw(output='mpl')

layout = {q[0]: 1, q[1]: 13, q[2]: 12, q[3]:2, q[4]:3}

backend = IBMQ.get_backend('ibmq_16_melbourne')
trans_ghz_mapped = transpile(circ, backend, initial_layout=layout)

job = execute(circ,
            backend,
            parameter_binds=[{phi: phi_val}
              for phi_val in phi_range],shots=shots)

job_monitor(job)

phi_range2 = phi_range
counts2 =[job.result().get_counts(i) for i in range(len(job.result().results))] 

print('\n')
print('Counts')
print(counts2)
print('\n')
print('Counts of 00000')
print(list(map(lambda c: 1/shots*c.get('00000', 0), counts2)))

job_id = job.job_id()
print('JOB ID: {}'.format(job_id))

#retrieved_job = backend.retrieve_job(job_id)
#retrieved_job.result().get_counts(qc)

Phi range
[0.0, 0.5235987755982988, 1.0471975511965976, 1.5707963267948966, 2.0943951023931953, 2.6179938779914944, 3.141592653589793, 3.665191429188092, 4.1887902047863905, 4.71238898038469, 5.235987755982989, 5.759586531581287]
Job Status: job has successfully run


Counts
[{'00001': 128, '10010': 1444, '11110': 93, '11101': 21, '00011': 135, '01100': 145, '11011': 30, '11001': 30, '11111': 21, '01011': 32, '10101': 45, '11000': 228, '01110': 136, '01101': 13, '01001': 34, '00010': 1697, '01010': 205, '10110': 127, '10100': 184, '10011': 142, '11100': 126, '10000': 962, '00101': 67, '11010': 205, '01000': 240, '00110': 152, '00100': 224, '01111': 18, '00000': 1085, '10001': 101, '00111': 79, '10111': 43}, {'00001': 148, '10010': 750, '11110': 94, '11101': 27, '00011': 153, '01100': 126, '11011': 45, '11001': 40, '11111': 18, '01011': 37, '10101': 42, '11000': 261, '01110': 156, '01101': 25, '01001': 42, '00010': 800, '01010': 250, '10110': 117, '10100': 120, '10011': 116, '11100': 86

In [4]:
#Run in real quantum computer - Transpile user CIRCUITO MODIFICATOOOOO
#Misura di tutti i qubit
n = 5
q = QuantumRegister(n, 'q')
c = ClassicalRegister(n, 'c') #n

circ = QuantumCircuit(q,c)
phi = Parameter('phi')

#1 circuito modificato
circ.h(q[0])
circ.cx(q[1], q[0])
circ.cx(q[1], q[2])
circ.cx(q[2], q[3])
circ.cx(q[3], q[4])  
circ.barrier()
circ.rz(phi, range(n))
circ.barrier()
circ.cx(q[3], q[4])
circ.cx(q[2], q[3])
circ.cx(q[1], q[2])
circ.cx(q[1], q[0])
circ.h(q[0])


circ.barrier()

for i in range(n):
    circ.measure(q[i], c[i]) 

print(circ.draw(output='latex_source',plot_barriers=False))

% \documentclass[preview]{standalone}
% If the image is too large to fit on this documentclass use
\documentclass[draft]{beamer}
% img_width = 10, img_depth = 21
\usepackage[size=custom,height=15,width=36,scale=0.7]{beamerposter}
% instead and customize the height and width (in cm) to fit.
% Large images may run out of memory quickly.
% To fix this use the LuaLaTeX compiler, which dynamically
% allocates memory.
\usepackage[braket, qm]{qcircuit}
\usepackage{amsmath}
\pdfmapfile{+sansmathaccent.map}
% \usepackage[landscape]{geometry}
% Comment out the above line if using the beamer documentclass.
\begin{document}
\begin{equation*}
    \Qcircuit @C=0.5em @R=1.0em @!R {
	 	\lstick{q_{0}: \ket{0}} & \gate{H} & \targ & \qw & \qw & \qw & \qw & \gate{R_z(phi)} & \qw & \qw & \qw & \qw & \targ & \gate{H} & \qw & \meter & \qw & \qw & \qw & \qw & \qw & \qw\\
	 	\lstick{q_{1}: \ket{0}} & \qw & \ctrl{-1} & \ctrl{1} & \qw & \qw & \qw & \gate{R_z(phi)} & \qw & \qw & \qw & \ctrl{1} & \ctrl{-1} & \qw &